In [1]:
# imports

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


"""
# -- run once only

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

"""

"\n# -- run once only\n\nnltk.download('punkt_tab')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('wordnet')\n\n"

In [2]:
# reading Database

anime = pd.read_csv("../archive/anime-dataset-2023.csv")

In [3]:
# necessary functions

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> list:
    tokens = word_tokenize(str(text).lower())
    lemmas = [lemmatizer.lemmatize(tok) for tok in tokens if tok.isalpha()]
    return " ".join(lemmas)


def tfidf(text_series):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text_series)
    feature_names = vectorizer.get_feature_names_out()

    return tfidf_matrix, feature_names

In [4]:
anime_synopsis_matrix = anime[['anime_id', 'Synopsis']].copy()

# lemmatizing
anime_synopsis_matrix['Synopsis'] = anime_synopsis_matrix['Synopsis'].map(lemmatize_text)

tfidf_matrix, feature_names = tfidf(anime_synopsis_matrix['Synopsis'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names).astype(np.float32)
tfidf_df['anime_id'] = anime_synopsis_matrix['anime_id']

In [5]:
tfidf_df.to_parquet('../data/Synopsis_processed.parquet', engine='pyarrow', index=False)